# Importando as bibliotecas

In [0]:
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql.functions import col, when, current_date, count, avg


# Pontos de Montagem (bronze, silver e gold)

In [0]:
# Montar o caminho para o contêiner bronze na Azure
mounts = dbutils.fs.mounts()
mounted_paths = [mount.mountPoint for mount in mounts]

if "/mnt/stdataengineeringmvp/bronze" not in mounted_paths:
    dbutils.fs.mount(
        source="wasbs://bronze@stdataengineeringmvp.blob.core.windows.net",
        mount_point="/mnt/stdataengineeringmvp/bronze",
        extra_configs={"fs.azure.account.key.stdataengineeringmvp.blob.core.windows.net": "[key-storage-account]"}
    )
else:
    print("O ponto de montagem BRONZE já existe.")

O ponto de montagem BRONZE já existe.


In [0]:
# Montar o caminho para o contêiner silver na Azure
mounts = dbutils.fs.mounts()
mounted_paths = [mount.mountPoint for mount in mounts]

if "/mnt/stdataengineeringmvp/silver" not in mounted_paths:
    dbutils.fs.mount(
        source="wasbs://silver@stdataengineeringmvp.blob.core.windows.net",
        mount_point="/mnt/stdataengineeringmvp/silver",
        extra_configs={"fs.azure.account.key.stdataengineeringmvp.blob.core.windows.net": "key-storage-account"}
    )
else:
    print("O ponto de montagem SILVER já existe.")

O ponto de montagem SILVER já existe.


In [0]:
# Montar o caminho para o contêiner ouro na Azure
mounts = dbutils.fs.mounts()
mounted_paths = [mount.mountPoint for mount in mounts]

if "/mnt/stdataengineeringmvp/gold" not in mounted_paths:
    dbutils.fs.mount(
        source="wasbs://gold@stdataengineeringmvp.blob.core.windows.net",
        mount_point="/mnt/stdataengineeringmvp/gold",
        extra_configs={"fs.azure.account.key.stdataengineeringmvp.blob.core.windows.net": "key-storage-account"}
    )
else:
    print("O ponto de montagem GOLD já existe.")

O ponto de montagem GOLD já existe.


# Preparando a Camada Bronze

In [0]:
# Carregando o arquivo do ENEM em um dataframe para iniciar o trabalho de análise da base, incluindo suporte para palavras com acentos
bronze_df_enem = spark.read.format('csv').options(header='true', infer_schema='true', delimiter=';', encoding='ISO-8859-1').load("dbfs:/mnt/stdataengineeringmvp/bronze/MICRODADOS_ENEM_2023.csv")

In [0]:
# Exibe a quantidade de linhas e colunas do dataframe ENEM
display(bronze_df_enem.count(), len(bronze_df_enem.columns))

3933955

76

In [0]:
# Criando o arquivo parquet para montagem da camada BRONZE
bronze_df_enem.write.format('delta').mode('overwrite').save('/mnt/stdataengineeringmvp/bronze/microdados_enem_2023_bronze')

In [0]:
# Cria o banco de dados se não existir
spark.sql('CREATE DATABASE IF NOT EXISTS BD_Microdados_Enem')

# Indicando o banco que deve ser usado para criação da tabela
spark.sql('USE DATABASE BD_Microdados_Enem')

# Criando a tabela e populando com os dados do arquivo delta da camada silver
spark.sql('''
CREATE TABLE IF NOT EXISTS BD_Microdados_Enem.tb_enem_bronze
AS SELECT * FROM delta.`/mnt/stdataengineeringmvp/bronze/microdados_enem_2023_bronze`
''')

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

# Preparando a Camada Silver


In [0]:
bronze_df_enem.dtypes

[('NU_INSCRICAO', 'string'),
 ('NU_ANO', 'string'),
 ('TP_FAIXA_ETARIA', 'string'),
 ('TP_SEXO', 'string'),
 ('TP_ESTADO_CIVIL', 'string'),
 ('TP_COR_RACA', 'string'),
 ('TP_NACIONALIDADE', 'string'),
 ('TP_ST_CONCLUSAO', 'string'),
 ('TP_ANO_CONCLUIU', 'string'),
 ('TP_ESCOLA', 'string'),
 ('TP_ENSINO', 'string'),
 ('IN_TREINEIRO', 'string'),
 ('CO_MUNICIPIO_ESC', 'string'),
 ('NO_MUNICIPIO_ESC', 'string'),
 ('CO_UF_ESC', 'string'),
 ('SG_UF_ESC', 'string'),
 ('TP_DEPENDENCIA_ADM_ESC', 'string'),
 ('TP_LOCALIZACAO_ESC', 'string'),
 ('TP_SIT_FUNC_ESC', 'string'),
 ('CO_MUNICIPIO_PROVA', 'string'),
 ('NO_MUNICIPIO_PROVA', 'string'),
 ('CO_UF_PROVA', 'string'),
 ('SG_UF_PROVA', 'string'),
 ('TP_PRESENCA_CN', 'string'),
 ('TP_PRESENCA_CH', 'string'),
 ('TP_PRESENCA_LC', 'string'),
 ('TP_PRESENCA_MT', 'string'),
 ('CO_PROVA_CN', 'string'),
 ('CO_PROVA_CH', 'string'),
 ('CO_PROVA_LC', 'string'),
 ('CO_PROVA_MT', 'string'),
 ('NU_NOTA_CN', 'string'),
 ('NU_NOTA_CH', 'string'),
 ('NU_NOTA_LC'

In [0]:
# Manter apenas os atributos que servirão para análise nesse MVP
selected_columns_df_enem = bronze_df_enem.select("NU_INSCRICAO", "NU_ANO", "TP_FAIXA_ETARIA", "TP_SEXO", "TP_ESTADO_CIVIL", "TP_COR_RACA", "TP_NACIONALIDADE", "TP_ST_CONCLUSAO", "TP_ANO_CONCLUIU", "TP_ESCOLA", "TP_ENSINO", "IN_TREINEIRO", "SG_UF_ESC", "TP_LOCALIZACAO_ESC", "NO_MUNICIPIO_PROVA", "SG_UF_PROVA", "NU_NOTA_MT", "NU_NOTA_LC", "Q001", "Q002", "Q006", "Q022", "Q024", "Q025")

In [0]:
display(selected_columns_df_enem.count(), len(selected_columns_df_enem.columns))

3933955

24

In [0]:
# Exibindo as primeiras linhas do dataframe para visualizar os dados
selected_columns_df_enem.head(n=2)

[Row(NU_INSCRICAO='210059085136', NU_ANO='2023', TP_FAIXA_ETARIA='14', TP_SEXO='M', TP_ESTADO_CIVIL='2', TP_COR_RACA='1', TP_NACIONALIDADE='1', TP_ST_CONCLUSAO='1', TP_ANO_CONCLUIU='17', TP_ESCOLA='1', TP_ENSINO=None, IN_TREINEIRO='0', SG_UF_ESC=None, TP_LOCALIZACAO_ESC=None, NO_MUNICIPIO_PROVA='Brasília', SG_UF_PROVA='DF', NU_NOTA_MT=None, NU_NOTA_LC=None, Q001='A', Q002='F', Q006='F', Q022='B', Q024='A', Q025='B'),
 Row(NU_INSCRICAO='210059527735', NU_ANO='2023', TP_FAIXA_ETARIA='12', TP_SEXO='M', TP_ESTADO_CIVIL='2', TP_COR_RACA='1', TP_NACIONALIDADE='0', TP_ST_CONCLUSAO='1', TP_ANO_CONCLUIU='16', TP_ESCOLA='1', TP_ENSINO=None, IN_TREINEIRO='0', SG_UF_ESC=None, TP_LOCALIZACAO_ESC=None, NO_MUNICIPIO_PROVA='Brasília', SG_UF_PROVA='DF', NU_NOTA_MT=None, NU_NOTA_LC=None, Q001='F', Q002='E', Q006='H', Q022='C', Q024='D', Q025='B')]

In [0]:
# Checando os valores nulos
display(selected_columns_df_enem.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in selected_columns_df_enem.columns]))

NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,SG_UF_ESC,TP_LOCALIZACAO_ESC,NO_MUNICIPIO_PROVA,SG_UF_PROVA,NU_NOTA_MT,NU_NOTA_LC,Q001,Q002,Q006,Q022,Q024,Q025
0,0,0,0,0,0,0,0,0,0,2594874,0,2975449,2975449,0,0,1241528,1111312,0,0,0,0,0,0


In [0]:
# Como um dos objetivos é identificar se as notas sofrem influência dos fatores socioeconômicos,
# serão mantidos apenas registros com ambas as notas preenchidas (matemática e língua).
# Além disso, foram excluídos registros nulos que identificam a região da escola do aluno.
filtered_df_enem = selected_columns_df_enem.filter("NU_NOTA_LC IS NOT NULL AND NU_NOTA_MT IS NOT NULL AND SG_UF_ESC IS NOT NULL AND TP_ENSINO IS NOT NULL")
#display(filtered_df_enem)


In [0]:
# Convertendo a tipagem de alguns atributos
filtered_df_enem = filtered_df_enem.withColumn("NU_INSCRICAO", col("NU_INSCRICAO").cast("bigint")) \
                                   .withColumn("NU_ANO", col("NU_ANO").cast("integer")) \
                                   .withColumn("TP_FAIXA_ETARIA", col("TP_FAIXA_ETARIA").cast("integer")) \
                                   .withColumn("TP_ENSINO", col("TP_ENSINO").cast("integer")) \
                                   .withColumn("NU_NOTA_LC", col("NU_NOTA_LC").cast("integer")) \
                                   .withColumn("NU_NOTA_MT", col("NU_NOTA_MT").cast("integer")) \
                                   .withColumn("TP_ESTADO_CIVIL", col("TP_ESTADO_CIVIL").cast("integer")) \
                                   .withColumn("TP_COR_RACA", col("TP_COR_RACA").cast("integer")) \
                                   .withColumn("TP_NACIONALIDADE", col("TP_NACIONALIDADE").cast("integer")) \
                                   .withColumn("TP_ST_CONCLUSAO", col("TP_ST_CONCLUSAO").cast("integer")) \
                                   .withColumn("TP_ANO_CONCLUIU", col("TP_ANO_CONCLUIU").cast("integer")) \
                                   .withColumn("TP_ESCOLA", col("TP_ESCOLA").cast("integer")) \
                                   .withColumn("TP_LOCALIZACAO_ESC", col("TP_LOCALIZACAO_ESC").cast("integer")) \
                                   .withColumn("IN_TREINEIRO", col("IN_TREINEIRO").cast("integer"))    

In [0]:
# Criando o arquivo parquet para montagem da camada SILVER
df_enem_silver = filtered_df_enem
df_enem_silver.write.format('delta').mode('overwrite').save('/mnt/stdataengineeringmvp/silver/microdados_enem_2023_silver')

# Criando o Banco de Dados


In [0]:

spark.sql('CREATE DATABASE IF NOT EXISTS BD_Microdados_Enem')

DataFrame[]

## Criando as Tabelas (silver)

In [0]:
# Indicando o banco que deve ser usado para criação da tabela
spark.sql('USE DATABASE BD_Microdados_Enem')

# Criando a tabela e populando com os dados do arquivo delta da camada silver
spark.sql('''
CREATE TABLE IF NOT EXISTS BD_Microdados_Enem.tb_enem_silver
AS SELECT * FROM delta.`/mnt/stdataengineeringmvp/silver/microdados_enem_2023_silver`
''')

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

# Preparando a Camada Gold


In [0]:
# Carregando um dataframe para alguns tratamentos finais para composição da camada GOLD
df_pre_gold = spark.read.format("delta").load("/mnt/stdataengineeringmvp/silver/microdados_enem_2023_silver")

In [0]:
display(df_pre_gold.head(n=2))

NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,SG_UF_ESC,TP_LOCALIZACAO_ESC,NO_MUNICIPIO_PROVA,SG_UF_PROVA,NU_NOTA_MT,NU_NOTA_LC,Q001,Q002,Q006,Q022,Q024,Q025
210060214087,2023,2,F,1,3,1,2,0,2,1,0,CE,1,Fortaleza,CE,466,507,D,D,C,D,A,B
210059980948,2023,3,F,1,3,1,2,0,2,1,0,CE,1,Quixadá,CE,338,446,B,B,B,B,A,A


In [0]:
# Renomear alguns atributos para facilitar a leitura e o entendimento
df_pre_gold = df_pre_gold.withColumnRenamed("Q001", "ESCOLARIDADE_PAI") \
                         .withColumnRenamed("Q002", "ESCOLARIDADE_MAE") \
                         .withColumnRenamed("Q006", "RENDA_FAMILIAR") \
                         .withColumnRenamed("Q022", "TEM_CELULAR") \
                         .withColumnRenamed("Q024", "TEM_COMPUTADOR") \
                         .withColumnRenamed("Q025", "TEM_INTERNET")
display(df_pre_gold.head(n=2))

NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,SG_UF_ESC,TP_LOCALIZACAO_ESC,NO_MUNICIPIO_PROVA,SG_UF_PROVA,NU_NOTA_MT,NU_NOTA_LC,ESCOLARIDADE_PAI,ESCOLARIDADE_MAE,RENDA_FAMILIAR,TEM_CELULAR,TEM_COMPUTADOR,TEM_INTERNET
210060214087,2023,2,F,1,3,1,2,0,2,1,0,CE,1,Fortaleza,CE,466,507,D,D,C,D,A,B
210059980948,2023,3,F,1,3,1,2,0,2,1,0,CE,1,Quixadá,CE,338,446,B,B,B,B,A,A


In [0]:
# As colunas "TEM_COMPUTADOR" e "TEM_CELULAR" possuem 5 categorias cada: 1 não e 4 sim's. Para esse MVP, basta manter apenas 2 valores de categorias (A=Não, B=Sim) 
# Substitui todos os valores C, D e E por B
df_pre_gold = df_pre_gold.withColumn("TEM_COMPUTADOR", 
                                     when(df_pre_gold["TEM_COMPUTADOR"].isin(["C", "D", "E"]), "B")
                                     .otherwise(df_pre_gold["TEM_COMPUTADOR"])) \
                        .withColumn("TEM_CELULAR", 
                                     when(df_pre_gold["TEM_CELULAR"].isin(["C", "D", "E"]), "B")
                                     .otherwise(df_pre_gold["TEM_CELULAR"]))

display(df_pre_gold.head(n=2))

NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,SG_UF_ESC,TP_LOCALIZACAO_ESC,NO_MUNICIPIO_PROVA,SG_UF_PROVA,NU_NOTA_MT,NU_NOTA_LC,ESCOLARIDADE_PAI,ESCOLARIDADE_MAE,RENDA_FAMILIAR,TEM_CELULAR,TEM_COMPUTADOR,TEM_INTERNET
210060214087,2023,2,F,1,3,1,2,0,2,1,0,CE,1,Fortaleza,CE,466,507,D,D,C,B,A,B
210059980948,2023,3,F,1,3,1,2,0,2,1,0,CE,1,Quixadá,CE,338,446,B,B,B,B,A,A


In [0]:
# Adicionar uma nova coluna para registrar a data da carga do dataset
df_pre_gold = df_pre_gold.withColumn("DT_CARGA", current_date())

In [0]:
# Criar o arquivo DELTA particionado por alguns atributos para acelerar as queries
# NU_ANO, TP_SEXO, TP_ESCOLA, SG_UF_ESC, SG_UF_PROVA
df_pre_gold.write.format('delta').mode('overwrite').option('mergeSchema', 'true').partitionBy('NU_ANO', 'TP_SEXO', 'TP_ESCOLA', 'SG_UF_ESC', 'SG_UF_PROVA').save('/mnt/stdataengineeringmvp/gold/microdados_enem_2023_gold')

## Criando as Tabelas (gold)

In [0]:
# Indicando o banco que deve ser usado para criação da tabela
spark.sql('USE DATABASE BD_Microdados_Enem')

# Criando a tabela e populando com os dados do arquivo delta da camada gold
spark.sql('''
CREATE TABLE IF NOT EXISTS BD_Microdados_Enem.tb_enem_gold
AS SELECT * FROM delta.`/mnt/stdataengineeringmvp/gold/microdados_enem_2023_gold`
''')

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

### Executando as consultas para respostas ao objetivo


1. EXISTE RELAÇÃO DA UF COM A MÉDIA DAS NOTAS? 

In [0]:
%sql
-- EXISTE VARIAÇÃO SIGNIFICANTE DO DESEMPENHO DAS NOTAS NOS DIFERENTES ESTADOS?
SELECT
  SG_UF_ESC AS UF_ESCOLA
  , round(avg(NU_NOTA_LC), 2) AS Media_LC
  , round(avg(NU_NOTA_MT), 2) AS Media_MT
FROM BD_Microdados_Enem.tb_enem_gold
GROUP BY
  SG_UF_ESC
ORDER BY Media_LC DESC

UF_ESCOLA,Media_LC,Media_MT
RS,542.33,575.8
SP,541.22,577.72
MG,541.1,592.52
DF,540.22,565.17
SC,539.39,573.73
RJ,537.7,572.83
PR,532.24,558.46
ES,532.19,570.94
RN,528.45,555.84
MS,520.42,543.2


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
-- EXISTE VARIAÇÃO SIGNIFICATIVA DO DESEMPENHO DO ALUNO COM A ESCOLARIDADE DO PAI?
WITH ESCOLARIDADE_MEDIA AS (
  SELECT 
    CASE
      WHEN ESCOLARIDADE_PAI = 'A' THEN 'Nunca estudou.'
      WHEN ESCOLARIDADE_PAI = 'B' THEN 'Não completou a 4ª série/5º ano do Ensino Fundamental.'
      WHEN ESCOLARIDADE_PAI = 'C' THEN 'Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental.'
      WHEN ESCOLARIDADE_PAI = 'D' THEN 'Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio.'
      WHEN ESCOLARIDADE_PAI = 'E' THEN 'Completou o Ensino Médio, mas não completou a Faculdade.'
      WHEN ESCOLARIDADE_PAI = 'F' THEN 'Completou a Faculdade, mas não completou a Pós-graduação.'
      WHEN ESCOLARIDADE_PAI = 'G' THEN 'Completou a Pós-graduação.'
      WHEN ESCOLARIDADE_PAI = 'H' THEN 'Não sei.'
    END AS ESCOLARIDADE_PAI

    , round(avg(NU_NOTA_LC), 2) AS Media_LC
    , round(avg(NU_NOTA_MT), 2) AS Media_MT
  FROM BD_Microdados_Enem.tb_enem_gold
  GROUP BY ESCOLARIDADE_PAI
)

SELECT *
FROM ESCOLARIDADE_MEDIA
ORDER BY Media_LC DESC



ESCOLARIDADE_PAI,Media_LC,Media_MT
Completou a Pós-graduação.,567.61,646.63
"Completou a Faculdade, mas não completou a Pós-graduação.",558.27,622.82
"Completou o Ensino Médio, mas não completou a Faculdade.",528.27,555.01
"Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio.",510.9,523.59
"Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental.",503.92,511.11
Não sei.,497.8,498.69
Não completou a 4ª série/5º ano do Ensino Fundamental.,489.29,487.59
Nunca estudou.,471.86,461.58


In [0]:
%sql
-- EXISTE VARIAÇÃO SIGNIFICATIVA DO DESEMPENHO DO ALUNO COM A ESCOLARIDADE DO MÃE?
WITH ESCOLARIDADE_MEDIA AS (
  SELECT 
    CASE
      WHEN ESCOLARIDADE_MAE = 'A' THEN 'Nunca estudou.'
      WHEN ESCOLARIDADE_MAE = 'B' THEN 'Não completou a 4ª série/5º ano do Ensino Fundamental.'
      WHEN ESCOLARIDADE_MAE = 'C' THEN 'Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental.'
      WHEN ESCOLARIDADE_MAE = 'D' THEN 'Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio.'
      WHEN ESCOLARIDADE_MAE = 'E' THEN 'Completou o Ensino Médio, mas não completou a Faculdade.'
      WHEN ESCOLARIDADE_MAE = 'F' THEN 'Completou a Faculdade, mas não completou a Pós-graduação.'
      WHEN ESCOLARIDADE_MAE = 'G' THEN 'Completou a Pós-graduação.'
      WHEN ESCOLARIDADE_MAE = 'H' THEN 'Não sei.'
    END AS ESCOLARIDADE_MAE

    , round(avg(NU_NOTA_LC), 2) AS Media_LC
    , round(avg(NU_NOTA_MT), 2) AS Media_MT
  FROM BD_Microdados_Enem.tb_enem_gold
  GROUP BY ESCOLARIDADE_MAE
)

SELECT *
FROM ESCOLARIDADE_MEDIA
ORDER BY Media_LC DESC

ESCOLARIDADE_MAE,Media_LC,Media_MT
Completou a Pós-graduação.,557.23,621.59
"Completou a Faculdade, mas não completou a Pós-graduação.",550.46,606.37
"Completou o Ensino Médio, mas não completou a Faculdade.",521.89,542.2
"Completou a 8ª série/9º ano do Ensino Fundamental, mas não completou o Ensino Médio.",501.97,505.83
"Completou a 4ª série/5º ano, mas não completou a 8ª série/9º ano do Ensino Fundamental.",493.72,492.08
Não sei.,483.18,484.4
Não completou a 4ª série/5º ano do Ensino Fundamental.,482.33,475.01
Nunca estudou.,465.51,452.73


In [0]:
%sql
-- EXISTE VARIAÇÃO SIGNIFICATIVA DO DESEMPENHO DO ALUNO COM A RENDA FAMILIAR?
WITH RENDA_FAMILIAR AS (
  SELECT 
    CASE
      WHEN RENDA_FAMILIAR = 'A' THEN 'Nenhuma Renda'
      WHEN RENDA_FAMILIAR = 'B' THEN 'Até R$ 1.320,00'
      WHEN RENDA_FAMILIAR = 'C' THEN 'De R$ 1.320,01 até R$ 1.980,00.'
      WHEN RENDA_FAMILIAR = 'D' THEN 'De R$ 1.980,01 até R$ 2.640,00.'
      WHEN RENDA_FAMILIAR = 'E' THEN 'De R$ 2.640,01 até R$ 3.300,00.'
      WHEN RENDA_FAMILIAR = 'F' THEN 'De R$ 3.300,01 até R$ 3.960,00.'
      WHEN RENDA_FAMILIAR = 'G' THEN 'De R$ 3.960,01 até R$ 5.280,00.'
      WHEN RENDA_FAMILIAR = 'H' THEN 'De R$ 5.280,01 até R$ 6.600,00.'
      WHEN RENDA_FAMILIAR = 'I' THEN 'De R$ 6.600,01 até R$ 7.920,00.'
      WHEN RENDA_FAMILIAR = 'J' THEN 'De R$ 7.920,01 até R$ 9.240,00.'
      WHEN RENDA_FAMILIAR = 'K' THEN 'De R$ 9.240,01 até R$ 10.560,00.'
      WHEN RENDA_FAMILIAR = 'L' THEN 'De R$ 10.560,01 até R$ 11.880,00.'
      WHEN RENDA_FAMILIAR = 'M' THEN 'De R$ 11.880,01 até R$ 13.200,00.'
      WHEN RENDA_FAMILIAR = 'N' THEN 'De R$ 13.200,01 até R$ 15.840,00.'
      WHEN RENDA_FAMILIAR = 'O' THEN 'De R$ 15.840,01 até R$19.800,00.'
      WHEN RENDA_FAMILIAR = 'P' THEN 'De R$ 19.800,01 até R$ 26.400,00.'      
      WHEN RENDA_FAMILIAR = 'Q' THEN 'Acima de R$ 26.400,00.'            
    END AS RENDA_FAMILIAR

    , round(avg(NU_NOTA_LC), 2) AS Media_LC
    , round(avg(NU_NOTA_MT), 2) AS Media_MT
  FROM BD_Microdados_Enem.tb_enem_gold
  GROUP BY RENDA_FAMILIAR
)

SELECT *
FROM RENDA_FAMILIAR
ORDER BY Media_LC DESC

RENDA_FAMILIAR,Media_LC,Media_MT
"Acima de R$ 26.400,00.",587.5,704.55
"De R$ 19.800,01 até R$ 26.400,00.",585.24,688.52
"De R$ 15.840,01 até R.800,00.",577.49,672.22
"De R$ 13.200,01 até R$ 15.840,00.",573.13,663.11
"De R$ 11.880,01 até R$ 13.200,00.",572.04,656.35
"De R$ 10.560,01 até R$ 11.880,00.",565.48,646.21
"De R$ 9.240,01 até R$ 10.560,00.",562.91,637.66
"De R$ 7.920,01 até R$ 9.240,00.",559.17,626.97
"De R$ 6.600,01 até R$ 7.920,00.",554.84,615.62
"De R$ 5.280,01 até R$ 6.600,00.",550.58,605.39


In [0]:
%sql
-- EXISTE VARIAÇÃO SIGNIFICATIVA DO DESEMPENHO DO ALUNO COM ACESSO A COMPUTADOR COM INTERNET?
WITH INFRAESTRUTURA_DESKTOP_INTERNET AS (
  SELECT 
    CASE
      WHEN TEM_COMPUTADOR = 'B' AND TEM_INTERNET = 'B' THEN 'Sim'
      ELSE 'Não'
    END AS DESKTOP_INTERNET
    , round(avg(NU_NOTA_LC), 2) AS Media_LC
    , round(avg(NU_NOTA_MT), 2) AS Media_MT
  FROM BD_Microdados_Enem.tb_enem_gold
  GROUP BY DESKTOP_INTERNET
)

SELECT *
FROM INFRAESTRUTURA_DESKTOP_INTERNET
ORDER BY Media_LC DESC

DESKTOP_INTERNET,Media_LC,Media_MT
Sim,545.39,593.65
Não,494.5,493.54


In [0]:
%sql
-- EXISTE VARIAÇÃO SIGNIFICATIVA DO DESEMPENHO DO ALUNO DE ESCOLA PÚBLICA E PRIVADA?
WITH TIPO_ESCOLA_ENS_MEDIO AS (
  SELECT 
    CASE
      --WHEN TP_ESCOLA = 1 THEN 'Não Respondeu'
      WHEN TP_ESCOLA = 2 THEN 'Pública'
      WHEN TP_ESCOLA = 3 THEN 'Privada'
    END AS TIPO_ESCOLA
    , round(avg(NU_NOTA_LC), 2) AS Media_LC
    , round(avg(NU_NOTA_MT), 2) AS Media_MT
  FROM BD_Microdados_Enem.tb_enem_gold
  WHERE TP_ESCOLA IN (2, 3)
  GROUP BY TIPO_ESCOLA
)

SELECT *
FROM TIPO_ESCOLA_ENS_MEDIO
ORDER BY Media_LC DESC

TIPO_ESCOLA,Media_LC,Media_MT
Privada,563.37,638.02
Pública,507.18,515.1


In [0]:
#filtered_df_enem.createOrReplaceTempView('tb_enem_2023')
#%sql
#select * from tb_enem_2023
#limit 5

---------------------------------------------------------------------------
ParseException                            Traceback (most recent call last)
File <command-4332324333280877>, line 3
      1 spark.sql('USE DATABASE bd_microdados_enem')
----> 3 spark.sql('''
      4 CREATE TABLE IF NOT EXISTS BD_Microdados_Enem.tb_enem_2023_attributes
      5 AS SELECT * FROM delta.'/mnt/stdataengineeringmvp/silver/microdados_enem_2023_silver`
      6 ''')

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/session.py:1820, in SparkSession.sql(self, sqlQuery, args, **kwargs)
   1815     else:
   1816         raise PySparkTypeError(
   18